# Анализ базы данных с помощью SQL #

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы и бросились создавать приложения для тех, кто любит читать.

Задача — необходимо проанализировать базу данных, в которой представлена информация о книгах, издательствах, авторах, а также пользовательские ценки и обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта компании, запустившей сервис для чтения книг по подписке.

Анализ базы данных включает в себя решение следующих задач:

* подсчет количества книг вышедших после 1 января 2000 года,

* подсчет количества обзоров и средней оценки для каждой книги,

* определение издательства, которое выпустило наибольшее число книг толще 50 страниц,

* определение автора с самой высокой средней оценкой книг,

* подсчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок.


<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подключение-к-базе-данных" data-toc-modified-id="Подключение-к-базе-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подключение к базе данных</a></span></li><li><span><a href="#Запросы-к-базе-данных" data-toc-modified-id="Запросы-к-базе-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Запросы к базе данных</a></span><ul class="toc-item"><li><span><a href="#Подсчет-количества-книг-вышедших-после-1-января-2000-года" data-toc-modified-id="Подсчет-количества-книг-вышедших-после-1-января-2000-года-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подсчет количества книг вышедших после 1 января 2000 года</a></span></li><li><span><a href="#Подсчет-количества-обзоров-и-средней-оценки-для-каждой-книги" data-toc-modified-id="Подсчет-количества-обзоров-и-средней-оценки-для-каждой-книги-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Подсчет количества обзоров и средней оценки для каждой книги</a></span></li><li><span><a href="#Определение-издательства,-которое-выпустило-наибольшее-число-книг-толще-50-страниц-(для-исключения-из-анализа-брошюр)" data-toc-modified-id="Определение-издательства,-которое-выпустило-наибольшее-число-книг-толще-50-страниц-(для-исключения-из-анализа-брошюр)-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Определение издательства, которое выпустило наибольшее число книг толще 50 страниц (для исключения из анализа брошюр)</a></span></li><li><span><a href="#Определение-автора-с-самой-высокой-средней-оценкой-книг-(учитываются-только-книги-с-50-и-более-оценками)" data-toc-modified-id="Определение-автора-с-самой-высокой-средней-оценкой-книг-(учитываются-только-книги-с-50-и-более-оценками)-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Определение автора с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками)</a></span></li><li><span><a href="#Подсчет-среднего-количества-обзоров-от-пользователей,-которые-поставили-больше-50-оценок" data-toc-modified-id="Подсчет-среднего-количества-обзоров-от-пользователей,-которые-поставили-больше-50-оценок-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Подсчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

## Подключение к базе данных

In [1]:
# импорт библиотек
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
# выполняем sql-запрос
query = '''SELECT *
           FROM books
           LIMIT 5;
        ''' 
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица `books` cодержит данные о книгах:

- `book_id` — идентификатор книги,
- `author_id` — идентификатор автора,
- `title` — название книги,
- `num_pages` — количество страниц,
- `publication_date` — дата публикации книги,
- `publisher_id` — идентификатор издателя.

In [3]:
# выполняем sql-запрос
query = '''SELECT *
           FROM authors
           LIMIT 5;
        ''' 
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица `authors` содержит данные об авторах:

- `author_id` — идентификатор автора,
- `author` — имя автора.

In [4]:
# выполняем sql-запрос
query = '''SELECT *
           FROM publishers
           LIMIT 5;
        ''' 
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица `publishers` содержит данные об издательствах:

- `publisher_id` — идентификатор издательства,
- `publisher` — название издательства.

In [5]:
# выполняем sql-запрос
query = '''SELECT *
           FROM ratings
           LIMIT 5;
        ''' 
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица `ratings` содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки,
- `book_id` — идентификатор книги,
- `username` — имя пользователя, оставившего оценку,
- `rating` — оценка книги.

In [6]:
# выполняем sql-запрос
query = '''SELECT *
           FROM reviews
           LIMIT 5;
        ''' 
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица `reviews` содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора,
- `book_id` — идентификатор книги,
- `username` — имя пользователя, написавшего обзор,
- `text` — текст обзора.

Мы получили данные из 5 таблиц, содержащих информацию о книгах, авторах, издательствах, пользовательских оценках и обзорах книг. 

## Запросы к базе данных

### Подсчет количества книг вышедших после 1 января 2000 года

In [7]:
query = '''

SELECT COUNT(book_id)
FROM books
WHERE publication_date > '01-01-2000';
        
        '''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Подсчет количества обзоров и средней оценки для каждой книги

In [8]:
query = '''

SELECT b.title AS book_title,
       COUNT(DISTINCT review_id) AS count_reviews,
       ROUND(AVG(rating),1) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS re ON b.book_id=re.book_id
LEFT JOIN ratings AS ra ON b.book_id=ra.book_id
GROUP BY b.book_id
ORDER BY avg_rating DESC,
         count_reviews DESC;
    
        '''
pd.io.sql.read_sql(query, con = engine)

,book_title,count_reviews,avg_rating
0,A Dirty Job (Grim Reaper #1),4,5.0
1,School's Out—Forever (Maximum Ride #2),3,5.0
2,Moneyball: The Art of Winning an Unfair Game,3,5.0
3,Crucial Conversations: Tools for Talking When ...,2,5.0
4,Misty of Chincoteague (Misty #1),2,5.0
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.3
996,Drowning Ruth,3,2.0
997,His Excellency: George Washington,2,2.0
998,Junky,2,2.0


Мы вывели количество обзоров и средний рейтинг для каждой книги.

### Определение издательства, которое выпустило наибольшее число книг толще 50 страниц (для исключения из анализа брошюр)

In [9]:
query = '''

SELECT p.publisher,
       COUNT(b.book_id) AS count_book
FROM publishers AS p
JOIN books AS b ON p.publisher_id=b.publisher_id
WHERE num_pages > 50
GROUP BY p.publisher
ORDER BY count_book DESC
LIMIT 1;
        
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher,count_book
0,Penguin Books,42


Издательство <b>Penguin Books</b> выпустило больше всего книг с количеством страниц больше 50.

### Определение автора с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками)

In [10]:
query = '''

SELECT a_r_avg.author AS author,
       AVG(a_r_avg.avg_rating) AS avg_rating
FROM (SELECT a.author AS author,
             b.book_id AS id,
             AVG(ra.rating) AS avg_rating,
             COUNT(ra.rating) AS count_rating
        FROM authors AS a
        INNER JOIN books AS b ON b.author_id = a.author_id
        INNER JOIN ratings AS ra ON ra.book_id = b.book_id
        GROUP BY author, id
        HAVING COUNT(ra.rating) >= 50) AS a_r_avg
GROUP BY author
ORDER BY avg_rating DESC
LIMIT 1;
        
        '''
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


<b>J.K. Rowling/Mary GrandPré</b> - автор с самой высокой средней оценкой книг (4.28)

### Подсчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
query = '''

WITH 
A AS 
(SELECT username,
        COUNT(DISTINCT(review_id))
FROM reviews       
WHERE username IN (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(DISTINCT(book_id)) > 50)
GROUP BY username) 
SELECT AVG(COUNT)
FROM A;
        '''
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


Среднее количество обзоров от пользователей, поставивших больше 50 оценок, составляет 24.3

## Вывод

* Мы проанализировали базу данных, включающую в себя информацию о книгах, авторах, издательствах, а также пользовательские оценки и обзоры книг


* Для каждой книги была получена информация о количестве обзоров и среднем рейтинге 


* В ходе работы мы выяснили, что после 1 января 2000 года вышло 819 книг


* Издательство, выпустившее больше всего книг - <b>Penguin Books</b>


* <b>J.K. Rowling/Mary GrandPré</b> - автор с самой высокой средней оценкой книг - 4.28


* Среднее количество обзоров от пользователей, поставивших больше 50 оценок, составляет 24.3